### Libraries

Install libraries

In [1]:
%cd "C:\Users\DELL\Documents\Python\AES"

C:\Users\DELL\Documents\Python\AES


In [2]:
# %pip install -r requirements.txt

Import libraries

In [3]:
import socket
import numpy as np
import cv2
from Crypto.Cipher import AES
from Crypto import Random
import os
import glob
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

C:\Users\DELL\Documents\Python\venv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll


### Client socket

In [4]:
HOST='192.168.137.1'
PORT=8000
WIDTH = 668
HEIGHT = 960
SC = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = (HOST, PORT)

### Helps function:
- Receive data
- Generate 128/192/256 bit key from any key
- Make folder to save datas
- Socket connection

In [5]:
def recv_all(sc, length):
    data = sc.recv(length)
    while True:
        miss_len_data = length - len(data)
        if miss_len_data==0:
            break
        data += sc.recv(miss_len_data)
    return data

def generateAESKey(key_string, len_key):
    '''Generate 128/192/256 bit key from any key'''
    key = bytes(key_string,"utf8")
    while len(key)<len_key:
        key = key + key
    return key[:len_key]

def make_folder(folder_name):
    path_f = os.path.abspath('')
    if os.path.isdir(path_f+"/"+folder_name) == False:
        os.mkdir(path_f+"/"+folder_name)
    list_folder = glob.glob(path_f+"/"+folder_name+"/*")
    max=0
    for i in list_folder:
        try:
            num = int(i.split("-")[1])
            if(max<num): max = num
        except: continue
    path = path_f+"/"+folder_name+"/data-"+str(max+1)
    os.mkdir(path)
    return path

# connect to client
def connect(sc, server_address):
    print("Connecting to the server...")
    while True:
        try:
            sc.connect(server_address)
            break
        except:
            ()
    print("[Connected]", server_address)
    return sc
    

### AES - ECB mode

In [6]:
LENGTH_IV = AES.block_size
def encode_AES(data, key, mode):
    if mode != AES.MODE_ECB:
        IV = Random.new().read(LENGTH_IV)
        cipher = AES.new(key, mode, IV)
        ciphertext =   IV + cipher.encrypt(data)
    else:
        cipher = AES.new(key, AES.MODE_ECB)
        ciphertext = cipher.encrypt(data)
    return ciphertext

def decode_AES(data_enc, key, mode): # decode AES
    if mode != AES.MODE_ECB:
        IV_rec = data_enc[:LENGTH_IV]
        cipher = AES.new(key, mode, IV_rec)
        plaintext = cipher.decrypt(data_enc[LENGTH_IV:])
    else:
        cipher = AES.new(key, AES.MODE_ECB)
        plaintext = cipher.decrypt(data_enc)
    return plaintext

### Function to receive image from server

In [7]:
def RecvImage(sc, key_AES, mode):
    print("Waiting for Server...")

    # create folder to save data
    save_path = make_folder("Client_images")
    len_data = HEIGHT*WIDTH*3 #size of a frame
    if mode != AES.MODE_ECB:
        len_data+=LENGTH_IV

    s_time = time.time()
    data_enc = recv_all(sc, len_data) # receive data    
    sc.send(b"Done")

    # Save Data
    f_data_enc = open(save_path+"/Data-received-from-server.enc", "wb")
    f_data_enc.write(data_enc)
    f_data_enc.close()
    rec_time = time.time()
    print(" - Received data: %d KB (%.3f seconds)" %(int(len(data_enc)/1024),rec_time-s_time))

    # Save Image befor AES-decode   
    if mode == AES.MODE_ECB:
        data_after_dec = np.frombuffer(data_enc, dtype = np.uint8)   # data encoded-> array
    else:
        data_after_dec = np.frombuffer(data_enc[LENGTH_IV:], dtype = np.uint8)
    data_after_dec = data_after_dec.reshape(HEIGHT, WIDTH, 3) # array encoded-> encoded image
    cv2.imwrite(save_path+"/image-before-decryption.bmp", data_after_dec)
    save_img_time = time.time()

    #Decode AES
    img_de = decode_AES(data_enc, key_AES, mode) # decode AES
    img_de = np.frombuffer(img_de,dtype = np.uint8) # byte to array
    img_de = img_de.reshape(HEIGHT, WIDTH, 3) # array to image
    aes_time =  time.time()
    print(" - Decoded AES. (%.3f seconds)" %(aes_time-save_img_time))
    
    # Save Image after AES-decode
    cv2.imwrite(save_path + "/AES-decrypted-image" + ".bmp",img_de)
    print("The data has been saved to the path: %s" %save_path)

    cv2.imshow("The image received from server", data_after_dec)
    cv2.imshow("AES decrypted image",img_de) # show decoded data
    cv2.waitKey(0)
    
    print("Done!")

### Main function

- **Enter key**

In [8]:
key = "Nhom-17"
key_AES = generateAESKey(key, 16)
print("Your AES key: ", key_AES)

Your AES key:  b'Nhom-17Nhom-17Nh'


- **Make a connection to the server**

In [9]:
SC = connect(SC, server_address)

Connecting to the server...
[Connected] ('192.168.137.1', 8000)


#### Transmit the image with ECB mode

In [10]:
RecvImage(SC, key_AES, AES.MODE_ECB)
 

Waiting for Server...
 - Received data: 1878 KB (16.105 seconds)
 - Decoded AES. (0.007 seconds)
The data has been saved to the path: C:\Users\DELL\Documents\Python\AES/Client_images/data-14
Done!


#### Transmit the image with CBC mode

In [11]:
RecvImage(SC, key_AES, AES.MODE_CBC)

Waiting for Server...
 - Received data: 1878 KB (0.003 seconds)
 - Decoded AES. (0.008 seconds)
The data has been saved to the path: C:\Users\DELL\Documents\Python\AES/Client_images/data-15
Done!


In [12]:
SC.close()